In [1]:
from rexerclient.rql import *
from rexerclient import client
import time


In [20]:
url = 'http://ad3946bfc322c4a23ac0f2546f1f6ea9-559200516.ap-south-1.elb.amazonaws.com/data'
#url = 'http://localhost:2425'
c = client.Client(url)
c.set_profile('unicorn', 12312, 'city', 'fremont')


ConnectionError: HTTPConnectionPool(host='ad3946bfc322c4a23ac0f2546f1f6ea9-559200516.ap-south-1.elb.amazonaws.com', port=80): Max retries exceeded with url: /data/set (Caused by ReadTimeoutError("HTTPConnectionPool(host='ad3946bfc322c4a23ac0f2546f1f6ea9-559200516.ap-south-1.elb.amazonaws.com', port=80): Read timed out. (read timeout=3)"))

In [16]:
def store_aggregates(c):
    # feature 1 -- no of views on Trail longer than 10 sec in the last 30 days
    q = Var('args').actions.apply(
        Ops.std.filter(where=it.action_type == 'view'),
        Ops.std.filter(where=it.metadata.watch_time >= 10),
        Ops.std.addField(name='groupkey', value=it.target_id),
    )

    options = {'aggregate_type': 'count', 'duration': 3600*24*30}
    c.store_aggregate('trail_num_10sec_views_30days_v2', q, options)

    # feature 2 -- for a userId Increase in the followers in last 7 & 28 days
    q = Var('args').actions.apply(
        Ops.std.filter(where=it.action_type == 'follow'),
        Ops.std.addField(name='groupkey', value=it.target_id),
    )

    options_7day = {'aggregate_type': 'count', 'duration': 3600*24*7}
    c.store_aggregate('user_num_follows_7day_v2', q, options_7day)

    options_28day = {'aggregate_type': 'count', 'duration': 3600*24*28}
    c.store_aggregate('user_num_follows_28day_v2', q, options_28day)

    # Feature 3 -- Avg-watchtime of a Trail for given country+OS+city_state+mobile_brand+gender at
    # {Sunday 11 to 12 am/current time} in the last 30 days

    q = Var('args').actions.apply(
        Ops.std.filter(where=it.action_type == 'view'),
        Ops.profile.addField(name='country', otype='user', oid=it.actor_id, key='country'),
        Ops.profile.addField(name='os', otype='user',oid=it.actor_id, key='os'),
        Ops.profile.addField(name='city_state',otype='user',oid=it.actor_id, key='city_state'),
        Ops.profile.addField(name='mobile_brand', otype='user', oid=it.actor_id, key='mobile_brand'),
        Ops.profile.addField(name='gender', otype='user', oid=it.actor_id, key='gender'),
        Ops.time.addDayOfWeek(name='day_of_week', timestamp=it.timestamp),
        Ops.time.addTimeBucketOfDay(name='time_bucket', timestamp=it.timestamp, bucket=3600),
        Ops.std.addField(name='groupkey', value=[
            it.target_id, it.country, it.os, it.city_state, it.mobile_brand, it.gender, it.day_of_week, it.time_bucket
        ]),
        Ops.std.addField(name='value', value=it.metadata.watch_time)
    )

    options = {'aggregate_type': 'average', 'duration': 3600*24*30}
    c.store_aggregate('complex_feature_v2', q, options)

    # feature  4 -- total views gained by a Trail on last 2 days for given city+gender+age_group
    q = Var('args').actions.apply(
        Ops.std.filter(where=it.action_type == 'view'),
        Ops.profile.addField(name='city', otype='user', oid=it.actor_id, key='city'),
        Ops.profile.addField(name='gender', otype='user', oid=it.actor_id, key='gender'),
        Ops.profile.addField(name='age_group', otype='user', oid=it.actor_id, key='age_group'),
        Ops.std.addField(name='groupkey', value=[it.target_id, it.city, it.gender, it.age_group]),
    )

    options = {'duration': 3600*24*2, 'aggregate_type': 'count', }
    c.store_aggregate('trail_view_by_city_gender_agegroup_2days_v2', q, options)

    # feature 5 - avg-watchtime of a user id  for creatorId in 2-hour window averaged over 30 days
    q = Var('args').actions.apply(
        Ops.std.filter(where=it.action_type == 'view'),
        Ops.profile.addField(name='creator_id', otype='user', oid=it.actor_id, key='creatorId'),
        Ops.time.addTimeBucketOfDay(name='time_bucket', timestamp=it.timestamp, bucket=2*3600),
        Ops.std.addField(name='groupkey', value=[it.actor_id, it.creator_id]),
        Ops.std.addField(name='value', value=it.metadata.watch_time),
    )
    options = {'aggregate_type': 'average', 'duration': 3600*24*30}
    c.store_aggregate('user_creator_avg_watchtime_by_2hour_windows_30days_v2', q, options)



In [17]:
def set_profiles(c, profiles):
    for profile in profiles:
       c.set_profile(profile['otype'], profile['oid'], profile['key'], profile['value'])


In [21]:
# initialize the client and point it towards the fennel servers
#url = 'http://ad3946bfc322c4a23ac0f2546f1f6ea9-559200516.ap-south-1.elb.amazonaws.com/data'
url = 'http://a3333e07f31614dd988930ad78f3f994-727687896.ap-south-1.elb.amazonaws.com/data'
#url = 'http://localhost:2425'
c = client.Client(url)

# now let's store the feature definitions (more technically known as "aggregates") using the client
store_aggregates(c)

# some features need some profile data. Since we don't have all the profile data yet, we will just
# store some of it manually
uid = 123
trell_id = 567
creator_id = 789
city = 'delhi'
gender = 1
age_group = '18_24'
country = 'IN'
os = 'android'
city_state = 'dehradun_uttarakhand'
mobile_brand = 'samsung'

set_profiles(c, [
    {'otype':'user', 'oid':uid, 'key':'city', 'value':city},
    {'otype':'user', 'oid':uid, 'key':'gender', 'value':gender},
    {'otype':'user', 'oid':uid, 'key':'age_group', 'value':age_group},
    {'otype':'user', 'oid':uid, 'key':'country', 'value':country},
    {'otype':'user', 'oid':uid, 'key':'os', 'value':os},
    {'otype':'user', 'oid':uid, 'key':'city_state', 'value':city_state},
    {'otype':'user', 'oid':uid, 'key':'mobile_brand', 'value':mobile_brand},
    {'otype':'trell', 'oid':trell_id, 'key':'creatorId', 'value':creator_id},
])


# now log a few actions - as these actions are logged, they will be picked up by stored aggregates
# in near realtime (with a lag of maybe a couple of minutes) and processed

# this action denotes that user with id: uid watched trell for 31 seconds
# note that any action has actor_type, actor_id, target_type, target_id, action_type, and timestamp
# but we can log any additional data with metadata, which can be an arbitrary jsonable data
c.log(actor_type='user', actor_id=uid, target_type='trail', target_id=trell_id, action_type='view',
      timestamp=int(time.time()), request_id=1, metadata={'watch_time': 31})

# another view action, but this time watch time is < 10 sec
c.log(actor_type='user', actor_id=uid, target_type='trail', target_id=trell_id, action_type='view',
      timestamp=int(time.time()), request_id=1, metadata={'watch_time': 7})

# now this is a follow action. Metadata is optional so we don't pass it here
c.log(actor_type='user', actor_id=uid, target_type='user', target_id=creator_id, action_type='follow',
      timestamp=int(time.time()), request_id=1)



ConnectionError: HTTPConnectionPool(host='a3333e07f31614dd988930ad78f3f994-727687896.ap-south-1.elb.amazonaws.com', port=80): Max retries exceeded with url: /data/store_aggregate (Caused by ReadTimeoutError("HTTPConnectionPool(host='a3333e07f31614dd988930ad78f3f994-727687896.ap-south-1.elb.amazonaws.com', port=80): Read timed out. (read timeout=3)"))

In [15]:
f1 = c.aggregate_value('trail_num_10sec_views_30days', trell_id)
print(f1)


0


In [66]:
c.fetch(action.ActionFetchRequest())

[ActionID: 1
 CustID: 1
 ActorID: 12312
 ActorType: "user"
 TargetID: 82342
 TargetType: "video"
 ActionType: "like"
 Timestamp: 1643689125
 RequestID: 523423,
 ActionID: 2
 CustID: 1
 ActorID: 12312
 ActorType: "user"
 TargetID: 82342
 TargetType: "video"
 ActionType: "like"
 Timestamp: 1643689149
 RequestID: 523423]

In [68]:

    

# Given a uid, what is their average watch time on any video they might have seen?
# when you log the action, you store video watch time in a field called 'action_value'
    
# this following query measures the total time spent by the user on any video in last x days
q1 = Var('args').actions.apply(
    Ops.std.filter(where=at.action_type == String('video_view'))).apply(        # filter to only 'like' actions
    Ops.std.addColumn(name='key', value=List(at.actor_id), increment=at.action_value)   # counter key is (uid)
)
options = models.aggregate.AggOptions()
options.duration = 3600*24*9  # 3 day rolling counters
c.store_aggregate('rolling_counter', 'user_video_like', q, options)


# measures total videos viewed by the user uid in the last x days
q2 = Var('args').actions.apply(
    Ops.std.addProfileColumn(otype='video', oid=at.target_id, key='category').apply(
    Ops.std.addProfileColumn(otype='user', oid=at.user_id, key='age').apply(
    Ops.std.filter(where=at.action_type == String('video_view'))).apply(        # filter to only 'like' actions
    Ops.std.addColumn(name='key', value=List(at.actor_id, at.gender, at.age, at.group_id), increment=1)   # counter key is (uid)
)
options = models.aggregate.AggOptions()
options.duration = 3600*24*9  # 3 day rolling counters
c.store_aggregate('rolling_counter', 'user_video_like', q, options)

find(q1) / find(q2)


In [69]:
req = models.aggregate.GetAggValueRequest()
req.agg_type = 'rolling_counter'
req.agg_name = 'user_video_like'
req.key.CopyFrom(value.List(value.Int(12312), value.Int(82342)))
c.aggregate_value(req)

Int: 0

In [28]:
lokal = 'http://a3333e07f31614dd988930ad78f3f994-727687896.ap-south-1.elb.amazonaws.com/data'
c = client.Client(lokal)

In [27]:
c.set_profile('unicorn', 123, 'city_new', 'bhiwani')

ConnectionError: HTTPConnectionPool(host='a3333e07f31614dd988930ad78f3f994-727687896.ap-south-1.elb.amazonaws.com', port=80): Max retries exceeded with url: /data/set (Caused by ReadTimeoutError("HTTPConnectionPool(host='a3333e07f31614dd988930ad78f3f994-727687896.ap-south-1.elb.amazonaws.com', port=80): Read timed out. (read timeout=3)"))